In [14]:
import pandas as pd
from os import walk
import os

In [21]:
!pip install yfinance --trusted-host pypi.org --trusted-host files.pythonhosted.org

In [22]:
!pip install pandas_ta --trusted-host pypi.org --trusted-host files.pythonhosted.org

  Using cached pandas_ta-0.2.45b.tar.gz (88 kB)
  Created wheel for pandas-ta: filename=pandas_ta-0.2.45b0-py3-none-any.whl size=175308 sha256=0834420411ea5981eca66c6564ac7aad6b5bdf6a919f6678636a94feeaa40c42
  Stored in directory: c:\users\ramesre\appdata\local\pip\cache\wheels\02\f6\96\09c888c53b185b0a86df45aa66f6abfc9d259e4c20edf13a5c
Successfully built pandas-ta


In [23]:
import pandas_ta as ta
import yfinance as yf

In [41]:
data = pd.read_csv("SNR_TI.csv")
data_raw = pd.read_csv("SNR.csv")
columns_raw = list(data_raw.columns)
print(columns_raw)
data.head()

['Unnamed: 0', 'date', 'open', 'high', 'low', 'close', 'adjClose', 'volume', 'unadjustedVolume', 'change', 'changePercent', 'vwap', 'label', 'changeOverTime']


,Unnamed: 0,Unnamed: 0.1,date,open,high,low,close,adjClose,volume,unadjustedVolume,...,STOCHk_14_3_3,STOCHd_14_3_3,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,WILLR_14,ADOSC_3_10,CCI_14_0.015,MOM_10
0,1257,1257,2016-05-02,10.80,11.08,10.71,11.07,7.030635,380900.0,380900.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1256,1256,2016-05-03,10.99,10.99,10.70,10.92,6.935369,217300.0,217300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1255,1255,2016-05-04,10.85,11.08,10.85,11.04,7.011583,288000.0,288000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1254,1254,2016-05-05,11.07,11.27,10.86,11.03,7.005231,452200.0,452200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1253,1253,2016-05-06,10.98,11.33,10.98,11.33,7.195762,500600.0,500600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
#data = data.sort_values(by="date", ascending=False)
## Creating lagged TI

columns_TI = list(set(data.columns) - set(columns_raw))
print(columns_TI)
data_lagged = data.copy()

trailing_window_size = 1

for window in range(1, trailing_window_size + 1):
    shifted = data[columns_TI].shift(window)
    shifted.columns = [x + "_lag" + str(window) for x in columns_TI]
    
    data_lagged = pd.concat((data_lagged, shifted), axis=1)
    
list(data_lagged.columns)
data_lagged.head()
    
data_lagged.to_csv("lag_test.csv")

['Unnamed: 0.1', 'MACDs_12_26_9', 'MACD_12_26_9', 'WILLR_14', 'RSI_14', 'STOCHk_14_3_3', 'SMA_20', 'ADOSC_3_10', 'MOM_10', 'FWMA_20', 'STOCHd_14_3_3', 'MACDh_12_26_9', 'CCI_14_0.015']


In [48]:
def create_lagged_features(df, columns, trailing_window = 1):

    data_lagged = df.copy()
    for window in range(1, trailing_window + 1):
        shifted = df[columns].shift(window)
        shifted.columns = [x + "_lag" + str(window) for x in columns]
        data_lagged = pd.concat((data_lagged, shifted), axis=1)

    return data_lagged


file = "SNR_TI.csv"

ticker = file.replace("_TI.csv", "")
data = pd.read_csv(file)

columns_TI = ['MACDs_12_26_9', 'MACD_12_26_9', 'WILLR_14', 'RSI_14', 'STOCHk_14_3_3',
              'SMA_20', 'ADOSC_3_10', 'MOM_10', 'FWMA_20', 'STOCHd_14_3_3',
              'MACDh_12_26_9', 'CCI_14_0.015'] # Edit this while creating the final code

data_TI_lagged = create_lagged_features(data,columns_TI)

raw_columns_analysis = ["close", "volume", "changePercent", "vwap", "changeOverTime"]
data_TI_raw_lagged = create_lagged_features(data_TI_lagged, raw_columns_analysis, 45)

data_TI_raw_lagged["ticker"] = ticker

data_TI_raw_lagged.to_csv("lag_test.csv")


In [18]:
from os import walk
a, b, filenames = next(walk(os.getcwd()))

In [20]:
b

['.ipynb_checkpoints']

In [9]:
def correlation(dataset, threshold, threshold_low):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if ((corr_matrix.iloc[i, j] >= threshold or corr_matrix.iloc[i, j] <= threshold_low ) and (corr_matrix.columns[j] not in col_corr)):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in dataset.columns:
                    del dataset[colname] # deleting the column from the dataset

    return dataset

In [12]:
data_removed = correlation(data,0.7,-0.7)
data_removed.to_csv("corr_removed.csv")